In [174]:
import ccxt
import pandas as pd
import numpy as np

def report_id_creator():
    timestamp = pd.Timestamp.now()
    id_year = timestamp.year 
    id_month = timestamp.month 
    id_day = timestamp.day 
    id_hour = timestamp.hour
    id_minute = timestamp.minute 
    id_second = timestamp.second

    # str(id_year)[-2:] 

    if id_month < 10:
        id_month = "0" + str(id_month)

    if id_day < 10:
        id_day = "0" + str(id_day)
    
    if id_hour < 10:
        id_hour = "0" + str(id_hour) 
    
    if id_minute < 10:
        id_minute = "0" + str(id_minute) 

    if id_second < 10:
        id_second = "0" + str(id_second) 

    detection_id_str = str(id_year) + "_" + str(id_month) + "_" + str(id_day)+ "_" + str(id_hour) + "_" + str(id_minute)  
    report_id = int(detection_id_str)
    
    return detection_id_str




def get_candle_spot(spot_market, num=5):
    # BTC/USDT için 1 saatlik mum verilerini çekme (örneğin son 50 saat)
    ohlcv = exchange.fetch_ohlcv(spot_market, timeframe='1h', limit=500)

    # Veriyi DataFrame'e dönüştürme
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Timestamp'i datetime'a dönüştürme (UTC olarak)
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

    # UTC'yi UTC+3'e (Europe/Istanbul) dönüştürme
    df['datetime'] = df['datetime'].dt.tz_localize('UTC').dt.tz_convert('Europe/Istanbul')

    # EMA Hesaplama (7 ve 21 periyotluk)
    df['EMA_7'] = df['close'].ewm(span=7, adjust=False).mean()
    df['EMA_21'] = df['close'].ewm(span=21, adjust=False).mean()

    # Al-Sat Sinyalleri
    # - EMA_7 eğer EMA_21 den büyükse 1 sinyalini veriri, 
    # - EMA_7 eğer EMA_21 den küçükse -1 sinyalini veriri.
    df['signal'] = np.where(df['EMA_7'] > df['EMA_21'], 1, -1)

    # DataFrame'i datetime sütununa göre indeksleme
    # df.set_index('datetime', inplace=True)


    df = df.sort_values("datetime")
    filtered_df = df[["datetime","EMA_7", "EMA_21", "signal"]].iloc[-num:]

    return filtered_df


def take_all_candle_spot(markets_usdt_spot):

    all_data = []

    for i in range(len(markets_usdt_spot)):

        symbol = markets_usdt_spot[i]["symbol"]
        # print(symbol)
        symbol_result = get_candle_spot(symbol, num=5)


        # if symbol_result.iloc[-3]["signal"] == symbol_result.iloc[-2]["signal"]:
        if symbol_result.iloc[-2]["signal"] == symbol_result.iloc[-1]["signal"]:
            variation = 0
            result = "Stabil"
        else:
            variation = 1
            if symbol_result.iloc[-1]["signal"] == 1:
                result = "Al"
            elif symbol_result.iloc[-1]["signal"] == -1:
                result = "Sat"
            else:
                result = "kod da bir hata var."

        new_data = {
        "Symbol": symbol,
        "Datetime_0": str(symbol_result.iloc[-5]["datetime"])[:-9],
        "Datetime_1": str(symbol_result.iloc[-4]["datetime"])[:-9],
        "Datetime_2": str(symbol_result.iloc[-3]["datetime"])[:-9],
        "Datetime_3": str(symbol_result.iloc[-2]["datetime"])[:-9],
        "Datetime_4": str(symbol_result.iloc[-1]["datetime"])[:-9],
        "Signal_0": symbol_result.iloc[-5]["signal"],
        "Signal_1": symbol_result.iloc[-4]["signal"],
        "Signal_2": symbol_result.iloc[-3]["signal"],
        "Signal_3": symbol_result.iloc[-2]["signal"],
        "Signal_4": symbol_result.iloc[-1]["signal"],
        "isVariation" : variation,
        "result" : result,
        }

        all_data.append(new_data)


    all_df_7_21 = pd.DataFrame(all_data)

    return all_df_7_21


In [175]:
# Binance API'sine bağlan
exchange = ccxt.binance({
    'apiKey': 'TVhlb29aiNZeugOZKZlxDOnAmmdOE7DWIFXblxQNnB8icZMyl4BWfDafbo9r9maW',
    'secret': 'Sc8hNaXvBZrwUkQKeRWt0kSqhzd33PGJs0bgBmgZtWKCHqaBqVQU4twklCaQgc52',
})

markets = exchange.fetch_markets()
markets_usdt_spot = [market for market in markets if (market['quote'] == 'USDT') & (market['spot'] == True) & (market["active"] == True)]
print(len(markets_usdt_spot))

397


In [176]:
all_df_7_21 = take_all_candle_spot(markets_usdt_spot)
desired_df_7_21 = all_df_7_21[all_df_7_21["isVariation"] == 1][["Symbol", "Datetime_0", "Datetime_1", "Datetime_2", "Datetime_3", "Datetime_4", "Signal_0", "Signal_1", "Signal_2", "Signal_3", "Signal_4", "result"]]
desired_df_7_21.reset_index(drop=True, inplace=True)

In [177]:
# DataFrame'i Excel dosyasına kaydetme
id = report_id_creator()
excel_filename = f"7_21_detection_{id}.xlsx"
desired_df_7_21.to_excel(excel_filename, index=False)
desired_df_7_21.to_excel("7_21_detection_referance.xlsx", index=False)
desired_df_7_21

,Symbol,Datetime_0,Datetime_1,Datetime_2,Datetime_3,Datetime_4,Signal_0,Signal_1,Signal_2,Signal_3,Signal_4,result
0,BNB/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,1,1,1,1,-1,Sat
1,TUSD/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,-1,-1,-1,-1,1,Al
2,CHZ/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,-1,-1,-1,-1,1,Al
3,STPT/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,1,1,1,1,-1,Sat
4,BADGER/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,-1,-1,-1,-1,1,Al
5,ALICE/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,1,1,1,1,-1,Sat
6,BETA/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,1,1,1,1,-1,Sat
7,ALPINE/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,-1,-1,-1,-1,1,Al
8,VIC/USDT,2025-02-12 04:00,2025-02-12 05:00,2025-02-12 06:00,2025-02-12 07:00,2025-02-12 08:00,1,1,1,1,-1,Sat


In [171]:
## Aldığın coinleri %1 stop koy %5 emir koy...


In [172]:
k = 7
bbb = get_candle_spot(markets_usdt_spot[k]["symbol"],10)
print(markets_usdt_spot[k]["symbol"])
bbb

XRP/USDT


,datetime,EMA_7,EMA_21,signal
490,2025-02-11 03:00:00+03:00,2.431380,2.424228,1
491,2025-02-11 04:00:00+03:00,2.437460,2.427089,1
492,2025-02-11 05:00:00+03:00,2.438920,2.428563,1
493,2025-02-11 06:00:00+03:00,2.439515,2.429721,1
494,2025-02-11 07:00:00+03:00,2.448961,2.434046,1
495,2025-02-11 08:00:00+03:00,2.465921,2.441569,1
496,2025-02-11 09:00:00+03:00,2.480041,2.448918,1
497,2025-02-11 10:00:00+03:00,2.485006,2.453552,1
498,2025-02-11 11:00:00+03:00,2.489479,2.458039,1
499,2025-02-11 12:00:00+03:00,2.488659,2.460599,1


In [179]:

bbb = get_candle_spot("ALPINE/USDT",num=10)
bbb

,datetime,EMA_7,EMA_21,signal
490,2025-02-11 23:00:00+03:00,1.028079,1.043572,-1
491,2025-02-12 00:00:00+03:00,1.029059,1.042520,-1
492,2025-02-12 01:00:00+03:00,1.029544,1.041473,-1
493,2025-02-12 02:00:00+03:00,1.030908,1.040885,-1
494,2025-02-12 03:00:00+03:00,1.034181,1.041168,-1
495,2025-02-12 04:00:00+03:00,1.037886,1.041880,-1
496,2025-02-12 05:00:00+03:00,1.039414,1.042073,-1
497,2025-02-12 06:00:00+03:00,1.041311,1.042521,-1
498,2025-02-12 07:00:00+03:00,1.042483,1.042837,-1
499,2025-02-12 08:00:00+03:00,1.045362,1.043852,1


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText

def send_email_with_attachment(subject, body, to_emails, file_path, file_name):
    # E-posta sunucusu yapılandırması
    from_email = "ikbalkaradeli@gmail.com"
    password = "M.i.k.2001g"  # Dikkat: Uzun vadede güvenliğiniz için uygulama bazlı parola kullanın.

    # E-posta başlığı
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = ', '.join(to_emails)
    msg['Subject'] = subject

    # E-posta gövdesi
    msg.attach(MIMEText(body, 'plain'))

    # Dosya ekleme
    attachment = open(file_path, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {file_name}")
    msg.attach(part)

    # E-posta gönderme
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login(from_email, password)
        server.sendmail(from_email, to_emails, msg.as_string().encode("utf-8"))
        server.quit()
        print("E-posta başarıyla gönderildi!")
    except Exception as e:
        print(f"E-posta gönderilirken bir hata oluştu: {e}")

# Kullanım örneği
send_email_with_attachment(
    subject="Deneme Excel Dosyası",
    body="Bu bir test e-postasıdır ve ekinde Excel dosyası vardır.",
    to_emails=["karadeli2001@hotmail.com"],
    file_path=f"/Users/ikbalkrdl/Desktop/Bitcoin-Bot/{excel_filename}",  # göndermek istediğiniz dosyanın yolu
    file_name="7_21_detection_2025_02_06_12_47"   # dosyanın adını yazın
)

E-posta gönderilirken bir hata oluştu: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 4fb4d7f45d1cf-5dcf1b7ae5asm619088a12.24 - gsmtp')
